In [1]:
# mnist demo (https://github.com/gnayuy/pythonlang/tree/master/tensorflow/mnist)
# by Yang (gnayuy@gmail.com) 4/17/2017
# this demo code combine the tensorflow.org tutorial and tensorflow dev summit 2017 tensorboard talk
#
# tensorboard --logdir="/tmp/mnist_demo" --port 6006

import os
import tensorflow as tf

In [2]:
# Load MNIST Data
LOGDIR = '/tmp/mnist_demo/'

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(train_dir=LOGDIR + 'data', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/mnist_demo/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/mnist_demo/data/train-labels-idx1-ubyte.gz


Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/mnist_demo/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/mnist_demo/data/t10k-labels-idx1-ubyte.gz


In [3]:
# Build a Multilayer Convolutional Network

def weight_variable(shape):
  return tf.Variable(tf.truncated_normal(shape, stddev=0.1), name="W")

def bias_variable(shape):
  return tf.Variable(tf.constant(0.1, shape=shape), name="B")

# Convolution and Pooling

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# conv

def conv_layer(input, size_in, size_out, name="conv"):
    with tf.name_scope(name):
        w = weight_variable([5, 5, size_in, size_out])
        b = bias_variable([size_out])
        conv = conv2d(input, w)
        act = tf.nn.relu(conv + b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return max_pool_2x2(act)
    
# fc

def fc_layer(input, size_in, size_out, name="fc"):
    with tf.name_scope(name):
        w = weight_variable([size_in, size_out])
        b = bias_variable([size_out])
        act = tf.nn.relu(tf.matmul(input, w) + b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act

In [12]:
#
learning_rate = 1E-4
tf.reset_default_graph()

# Setup placeholders, and reshape the data
x = tf.placeholder(tf.float32, shape=[None, 784], name="x")
x_image = tf.reshape(x, [-1, 28, 28, 1])
tf.summary.image('input', x_image, 3)
y = tf.placeholder(tf.float32, shape=[None, 10], name="labels")

# use 2 conv
conv1 = conv_layer(x_image, 1, 32, "conv1")
conv_out = conv_layer(conv1, 32, 64, "conv2")

flattened = tf.reshape(conv_out, [-1, 7 * 7 * 64])

# use 2 fc
fc1 = fc_layer(flattened, 7 * 7 * 64, 1024, "fc1")
embedding_input = fc1
embedding_size = 1024

# Dropout
keep_prob = tf.placeholder(tf.float32)
fc1_drop = tf.nn.dropout(fc1, keep_prob)

y_conv = fc_layer(fc1_drop, 1024, 10, "fc2")

# Train and Evaluate the Model
with tf.name_scope("xent"):
    xent = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y), name="xent")
    tf.summary.scalar("xent", xent)
    
with tf.name_scope("train"):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(xent)
        
with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("accuracy", accuracy)

summ = tf.summary.merge_all()
            
embedding = tf.Variable(tf.zeros([1024, embedding_size]), name="test_embedding")
assignment = embedding.assign(embedding_input)
saver = tf.train.Saver()

with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(LOGDIR + 'mnist demo')
    writer.add_graph(session.graph)

    config = tf.contrib.tensorboard.plugins.projector.ProjectorConfig()
    embedding_config = config.embeddings.add()
    embedding_config.tensor_name = embedding.name
    embedding_config.sprite.image_path = LOGDIR + 'sprite_1024.png'
    embedding_config.metadata_path = LOGDIR + 'labels_1024.tsv'
    # Specify the width and height of a single thumbnail.
    embedding_config.sprite.single_image_dim.extend([28, 28])
    tf.contrib.tensorboard.plugins.projector.visualize_embeddings(writer, config)
    
    for i in range(2000):
        batch = mnist.train.next_batch(50)
        if i%100 == 0:
            [train_accuracy, s] = session.run([accuracy, summ],feed_dict={x:batch[0], y: batch[1], keep_prob: 1.0})
            writer.add_summary(s, i)
            print("step %d, training accuracy %g"%(i, train_accuracy))
            if i % 500 == 0:
                session.run(assignment, feed_dict={x: mnist.test.images[:1024], y: mnist.test.labels[:1024]})
                saver.save(session, os.path.join(LOGDIR, "model.ckpt"), i)
        train_step.run(feed_dict={x: batch[0], y: batch[1], keep_prob: 0.5})

    print("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.12


step 100, training accuracy 0.08


step 200, training accuracy 0.06


step 300, training accuracy 0.08


step 400, training accuracy 0.1


...

step 1600, training accuracy 0.94


step 1700, training accuracy 0.96


step 1800, training accuracy 0.98


step 1900, training accuracy 1


test accuracy 0.9742
